# Vorlesung 11-2

In [1]:
using Interact
using Plots
using LaTeXStrings
using LinearAlgebra
using Random
using Roots
using Printf
using Statistics

col = get_color_palette(:auto, plot_color(:white));

WebIO._IJuliaInit()

## Lagrange'sche Form der Polynominterpolation

mit zweiter baryzentrischer Formel

In [2]:
mutable struct Lagrange{T<:AbstractFloat}
    n             # order of polynomial
    x::Vector{T}  # nodes 
    λ::Vector{T}  # weights for barycentric formula
    f::Vector{T}  # f-values
    
    function Lagrange(x::Vector{T}) where T<:AbstractFloat
    # construction of weights
        n, λ = length(x), ones(T, size(x))
        for k = 1:n
            d = x[1:k-1] .- x[k]
            λ[1:k-1] ./=  d
            λ[k] /= prod(-d)
        end
        return new{T}(n, x, λ, zeros(T, size(x)))
    end
end

function LagrangeEval(p::Lagrange{T}, x::AbstractArray{T}) where {T<:AbstractFloat}
    num = den = zeros(T, size(x))
    for j=1:p.n                                         # 2nd barycentric formula
        num += (c = p.λ[j]./(x .- p.x[j]))*p.f[j]
        den += c
    end
    y = num./den
    ind = isnan.(y); y[ind] = p.f[indexin(x, p.x)[ind]] # evaluation at the nodes themselves
    return y
end

setf!(p::Lagrange{T}, f::Vector{T}) where {T<:AbstractFloat} = (p.f = f; p)

setf!(p::Lagrange{T}, f::Function)  where {T<:AbstractFloat} = setf!(p,f.(p.x))

Lagrange(x::Vector{T}, f) where {T<:AbstractFloat} = setf!(Lagrange(x), f)

LagrangeEval(p::Lagrange{T}, x::T) where {T<:AbstractFloat} = LagrangeEval(p, [x])[1]

(p::Lagrange)(x) = LagrangeEval(p, x) # enables calls of the form p(x)

## Newton'sche Form der Polynominterpolation

In [3]:
# Newton'sche Form der Polynominterpolation

struct Newton{T<:AbstractFloat}
    n::Integer    # order of polynomial
    x::Vector{T}  # nodes 
    d::Vector{T}  # divided differences
    f::Vector{T}  # f-values
    
    function Newton(x::Vector{T}, f::Vector{T}) where T<:AbstractFloat
    # construction of devided differences
        n, t, d = length(x), copy(f), zeros(T, size(x))
        for j=1:n
            for k=j-1:-1:1
                t[k] = (t[k+1]-t[k])/(x[j]-x[k])
            end
            d[j] = t[1]
        end
        return new{T}(n, x, d, f)
    end
end

function NewtonEval(p::Newton{T},x::AbstractArray{T}) where {T<:AbstractFloat}
    y = p.d[p.n]*ones(T, size(x))
    for k=p.n-1:-1:1                  # Horner-Schema
        y = y.*(x .- p.x[k]) .+ p.d[k]
    end
    return y
end

Newton(x::Vector{T},f::Function) where {T<:AbstractFloat} = Newton(x,f.(x))

NewtonEval(p::Newton{T}, x::T) where {T<:AbstractFloat} = NewtonEval(p,[x])[1]

(p::Newton)(x) = NewtonEval(p, x) # enables calls of the form p(x)

### Zwei Standard-Familien von Stützstellen

In [4]:
ChebyshevKnots(a, b, n) = (a+b)/2 .+ (a-b)/2*cos.(range(0,π,length=n+1))
EquidistantKnots(a, b, n) = range(a, b, length=n+1) |> collect;

## Beispiel: Runge-Phänomen für äquidistante Knoten

### Vorbereitung

In [9]:
# Runge-Funktion

α = 5
f(x) = 1 ./(1+α^2*x.^2)

xx = range(-1, 1, length=2000) # Punkte zum Auswerten von Fehlern und für die Plots

# Konvergenzkonstante im Satz von Bernstein

ρ = √(1+1/α^2)+1/α

# Konvergenzabszisse s für den äquidistanten Fall
# vgl. E. Hairer, G. Wanner, Introduction à l’Analyse Numérique, Université de Genève, 2005, S. 30-33
# http://www.unige.ch/~hairer/poly/poly.pdf

r1(x) = -2+2*x*atan(1/x)+log(1+x^2)
r2(x) = -2-(-1+x)*log(1-x)+(1+x)*log(1+x)-r1(1/α)
s = fzero(r2,0.0,0.99) 

0.7266768604776682

### Interaktiver Plot

In [10]:
@manipulate for n=10:71, knots=Dict("Chebyshev" => ChebyshevKnots, "equidistant" => EquidistantKnots)
    x = knots(-1,1,n)
    y = f.(x)
    yy = Lagrange(x,y)(xx)
    plot(xx,yy, linewidth = 2, ylims = (-0.2,1.2), legend = false, xticks = -1:0.25:1)
    (knots == EquidistantKnots) && plot!([-s s;  -s s],[-0.2,1.2], linewidth=4, color=col[5])
    scatter!(x,y, title = @sprintf("\nInterpolation in %s",knots), xlabel = L"$x$", ylabel = L"$p(x)$", 
        color = col[2])
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["n"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 71, :min => 10, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Int64} with 2 listeners. Value:
40, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/Folkmar/.julia/packages/Knockout/3gjS1/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/Folkmar/.julia/packages/Knockout/3gjS1/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/Folkmar/.julia/packages/InteractBase/o1I6G/src/../assets/all.js"), Asset("css", nothing, "/Users/Folkmar/.julia/packages/InteractBase/o1I6G/src/../assets/style.css"), Asset("css", nothing, "/Users/Folkmar/.julia/packages/Interact/pVyN2/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000147465330, Task (runnable) @0x0000000147465330), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"35\",\"36\",\"37\",\"38\",\"39\",\"40\",\"41\",\"42\",\"43\",\"44\",\"45\",\"46\",\"47\

## Illustration des Bernstein'schen Konvergenzsatzes für Tschebyscheff-Knoten

In [11]:
rg, fit = 10:4:200, 1:8
E = zeros(maximum(rg))
@manipulate for method = Dict("Lagrange" => Lagrange, "Newton" => Newton), 
    clipped = Dict("clipped" => true, "fill" => false)
    # Bestimmung des Interpolationsfehlers
    for n in rg
        p = method(ChebyshevKnots(-1,1,n),f)
        E[n] = norm(f.(xx)-p(xx),Inf)
    end
    # Fit der theoretischen Vorhersage: exponentielle Konvergenz
    c = mean(E[rg[fit]].*(ρ.^rg[fit]))
    # Plot
    plot(rg,c*ρ.^(-rg), yaxis=:log, linewidth = 2, label = "theory")
    clipped && plot!(ylims = (1e-16,1e0)) 
    scatter!(rg,E[rg], label = @sprintf("%s", method), title = "\nKonvergenzplot\n", xticks = 0:25:200)
    scatter!(ylabel = L"interpolation error $E_n = ||f-p_n||_\infty$")
    scatter!(xlabel = L"polynomial degree $n$")
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["method"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text : key")))], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}")))], Dict{Symbol, Any}(:attributes => ("data-bind" => "foreach : options_js"), :className => "buttons has-addons is-centered"))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol, Any}())], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("options_js" => (Observable{Vector{OrderedDict}} with 1 listeners. Value:
OrderedDict[OrderedDict{String, Any}("key" => "Lagrange", "val" => 1, "id" => "iddgxlnkIn"), OrderedDict{String, Any}("key" => "Newton", "val" => 2, "id" => "id8KiG4mEi")], nothing), "index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/Folkmar/.julia/packages/Knockout/3gjS1/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/Folkmar/.julia/packages/Knockout/3gjS1/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/Folkmar/.julia/packages/InteractBase/o1I6G/src/../assets/all.js"), Asset("css", nothing, "/Users/Folkmar/.julia/packages/InteractBase/o1I6G/src/../assets/style.css"), Asset("css", nothing, "/Users/Folkmar/.julia/packages/Interact/pVyN2/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("options_js" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"options_js\"]()) ? (this.valueFromJulia[\"options_js\"]=true, this.model[\"options_js\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000010f951330, Task (runnable) @0x000000010f951330), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var 

## Illustration der numerischen Instabilität der Newton-Form für größeren Polynomgrad

In [12]:
@manipulate for n=1:100, method = Dict("Lagrange" => Lagrange, "Newton" => Newton), 
                IEEE = Dict("double" => Float64, "single" => Float32), 
                clipped = Dict("clipped" => true, "fill" => false)
    x = IEEE.(ChebyshevKnots(-1,1,n))
    y = f.(x)
    yy = method(x,y)(IEEE.(xx))
    plot(xx, yy, linewidth = 2, xlabel = L"$x$", ylabel = L"$p(x)$", xticks = -1:0.25:1)
    clipped && plot!(ylims = (-0.2,1.2))
    scatter!(x, y, legend = false, title = @sprintf("\n%s{%s}-Form der Interpolation\n", method, eltype(x)))
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["n"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 100, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Int64} with 2 listeners. Value:
50, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/Folkmar/.julia/packages/Knockout/3gjS1/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/Folkmar/.julia/packages/Knockout/3gjS1/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/Folkmar/.julia/packages/InteractBase/o1I6G/src/../assets/all.js"), Asset("css", nothing, "/Users/Folkmar/.julia/packages/InteractBase/o1I6G/src/../assets/style.css"), Asset("css", nothing, "/Users/Folkmar/.julia/packages/Interact/pVyN2/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000110ded110, Task (runnable) @0x0000000110ded110), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"35\",\"36\",\"37\",\"38\",\"39\",